In [ ]:
from itertools import pairwise
import pandas as pd
import numpy as np
from scipy.integrate import trapezoid as integrate_trapezoid
import matplotlib.pyplot as plt
from airfoil import (
    Airfoil,
    WingSegment,
    Decomposer
)
from airfoil.wing import (
    angle_degrees_to_slope,
    mirror,
    auto_piecewise,
    auto_interpolate,
    ellipse_quadrant,
    calculated_wing_cube_loading,
    create_airfoil_sampler,
)
from airfoil.util.linestring_helpers import resample_shapes
from airfoil.util.array_helpers import create_array_interpolator
import pyvista as pv
XPS_FOAM_DENSITY = 40 # kg/m**3

In [ ]:
elevator_sections_at = np.array([
    0,
    25,
    150,
])
elevator_section = mirror(create_airfoil_sampler(
    airfoil         = lambda _: Airfoil.from_naca_designation("0012",100),
    leading_edge    = auto_interpolate([
        [  0,   0],
        [ 25,   0],
        [150, -20],
    ]),
    dihedral        = lambda _: 0,
    chord           = auto_interpolate([
        [  0, 120],
        [ 25, 120],
        [150, 100],
    ]),
    washout         = lambda _: 0,
    rotation_center = lambda _: 0,
))
elevator_segments = [
    WingSegment(
        left   = elevator_section(a),
        right  = elevator_section(b),
        length = b-a,
    )
    for a,b
    in pairwise(elevator_sections_at)
]

In [ ]:
rudder_sections_at = np.array([
    0,
    25,
    150,
])
rudder_section = mirror(create_airfoil_sampler(
    airfoil         = lambda _: Airfoil.from_naca_designation("0012",100),
    leading_edge    = auto_interpolate([
        [  0,   0],
        [ 25,   0],
        [150, -20],
    ]),
    dihedral        = lambda _: 0,
    chord           = auto_interpolate([
        [  0, 120],
        [ 25, 120],
        [150, 100],
    ]),
    washout         = lambda _: 0,
    rotation_center = lambda _: 0,
))
rudder_segments = [
    WingSegment(
        left   = rudder_section(a),
        right  = rudder_section(b),
        length = b-a,
    )
    for a,b
    in pairwise(rudder_sections_at)
]

In [ ]:
wing_sections_at = np.array([
    25,
    162.5,
    300,
    400,
    500,
    600,
    630,
    660,
    680,
    690,
])
leading_edge = mirror(auto_piecewise([
    ( 25, lambda x: 0),
    (300, lambda x: -x*angle_degrees_to_slope(5)),
    (600, lambda x: -x*angle_degrees_to_slope(3)),
    (700, lambda x: -x*angle_degrees_to_slope(3)+ellipse_quadrant(100,50,x))
]))
trailing_edge = mirror(auto_piecewise([
    ( 25.0, lambda x: -150),
    (162.5, lambda x: x*angle_degrees_to_slope( 5)),
    (650.0, lambda x: x*angle_degrees_to_slope(-2)),
    (700.0, lambda x: x*angle_degrees_to_slope(-2)-ellipse_quadrant(50,10,x))
]))
chord = lambda x: leading_edge(x)-trailing_edge(x)
dihedral = mirror(auto_piecewise([
    ( 25, lambda _: 0),
    (500, lambda x: x * angle_degrees_to_slope( 1)),
    (700, lambda x: x * angle_degrees_to_slope(15)),
]))
washout = mirror(auto_interpolate([
    [  0,  0],
    [162.5,  0],
    [400, -4],
    [700, -4],
]))
hinge_line = mirror(lambda x: np.where((x<300) | (x>600), np.nan, trailing_edge(x)+chord(x)*0.2))

wing_airfoil_shape_list = [
    (  0.0, Airfoil.from_airfoiltools_website("fx63137-il", cache_dir="./data/cache/").with_scale((100,100)).points),
    (400.0, Airfoil.from_airfoiltools_website("fx63137-il", cache_dir="./data/cache/").with_scale((100,100)).points),
    (500.0, Airfoil.from_naca_designation("23012",chord_length=100).with_translation((0,3)).points),
    (700.0, Airfoil.from_naca_designation("23112",chord_length=100).with_translation((0,3)).points),
]
wing_airfoil_shape = mirror(create_array_interpolator([
    (offset,resampled)
    for offset, resampled
    in zip(
        [i for i,_ in wing_airfoil_shape_list],
        resample_shapes(
            [j for _,j in wing_airfoil_shape_list],
            deflection_angle_split_deg=90,
        )
    )
]))
wing_airfoil = create_airfoil_sampler(
    airfoil         = lambda x: Airfoil(wing_airfoil_shape(x)),
    leading_edge    = leading_edge,
    dihedral        = dihedral,
    chord           = chord,
    washout         = washout,
    rotation_center = lambda x: chord(x)*0.25
)
wing_segments  = [
    WingSegment(
        wing_airfoil(sla),
        wing_airfoil(slb),
        slb-sla
    ) for sla, slb in pairwise(wing_sections_at[1:])
]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(15,4), sharex=True)

x = np.linspace(-700,700,400)

ax1.plot(x,  leading_edge(x))
ax1.plot(x, trailing_edge(x))
ax1.plot(x, hinge_line(x))
ax1.set_aspect("equal")

sar = (-wing_sections_at)[::-1].tolist()+wing_sections_at.tolist()
ax2.plot(sar, np.array([wing_airfoil_shape(xi)[:,1].max()/100*chord(xi)+dihedral(xi) for xi in sar]))
ax2.plot(sar, np.array([wing_airfoil_shape(xi)[:,1].min()/100*chord(xi)+dihedral(xi) for xi in sar]))
ax2.set_aspect("equal")

for section in wing_sections_at:
    ax1.axvline(section,linestyle=":",c="r",linewidth=1)
    ax1.axvline(-section,linestyle=":",c="r",linewidth=1)

In [ ]:
pt = pv.Plotter()
wing_segment_meshes= WingSegment.to_meshes(wing_segments, add_mirrored=True)
for m in wing_segment_meshes:
    pt.add_mesh(
        m.rotate_x(-6).translate((0,0,60)),
        #pbr=True, 
        smooth_shading=True,
        split_sharp_edges=True,
        feature_angle=70,
        roughness=0.1
    )
elevator_segment_meshes = WingSegment.to_meshes(elevator_segments, add_mirrored=True)
for m in elevator_segment_meshes:
    pt.add_mesh(
        m.translate((0,400,0)),
    )
rudder_segment_meshes = WingSegment.to_meshes(rudder_segments)
for m in rudder_segment_meshes:
    pt.add_mesh(
        m.rotate_y(-90).translate((0,400,0))
    )
cl = 600
pt.add_mesh(pv.Cylinder((0,cl/2,0),direction=(0,1,0),radius=8,height=cl).translate((0,-180,0)))
pt.show()

In [ ]:
wing_volume = sum(item.volume for item in wing_segment_meshes)
wing_area = integrate_trapezoid(
    np.array([leading_edge(xi)-trailing_edge(xi) for xi in x]),
    x,
)
wing_area_m2      = wing_area/1000**2
wing_span         = np.max(x)-np.min(x)
mean_chord        = wing_area/wing_span
aspect_ratio      = wing_span/mean_chord
total_aircraft_mass_estimate     = 1.0
wing_cube_loading = calculated_wing_cube_loading(total_aircraft_mass_estimate,wing_area_m2)
wing_mass         = wing_volume/1000**3*XPS_FOAM_DENSITY *1000

print(f"""
{wing_span = :.1f}
{mean_chord = :.1f}
{aspect_ratio = :.1f}
{wing_area_m2 = :.3f}
{wing_mass = :.0f} g
{total_aircraft_mass_estimate = :.1f} kg
{wing_cube_loading = :.1f}
""")

In [ ]:
pt.export_gltf("./data/outputs/2025 06 27 Glider wing Design Again.gltf")